# Intermovie - projet 6 
Utilisation de dataframes pour manipuler des données. Exportation en CSV pour chaque dataframe créé. 

## Import des librairies

In [ ]:
import pandas as pd

from my_timer import MyTimer
timer = MyTimer()

%load_ext autoreload
%autoreload 2

## 4. La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît
### Import des datas initiaux :

In [2]:
timer.start()

name_basics = pd.read_csv("../movies_dataset/name.basics.tsv", sep='\t', index_col='nconst', usecols=['nconst', 'primaryName', 'primaryProfession'])
print('Name basics imported.')

title_principals = pd.read_csv("../movies_dataset/title.principals.tsv", sep='\t', index_col='tconst', usecols=['nconst', 'tconst', 'category'])
print('Title principals imported.')

title_ratings = pd.read_csv("../movies_dataset/title.ratings.tsv", sep='\t', index_col='tconst', usecols=['tconst', 'averageRating'])
print('Title Ratings imported.')

title_basics = pd.read_csv("../movies_dataset/title.basics.tsv", sep='\t', index_col='tconst', usecols=['tconst', 'titleType'])
print('Title Basics imported.')

timer.stop("Chargement des csv :")

Name basics imported.
Title principals imported.
Title Ratings imported.
Title Basics imported.
Chargement des csv :
Elapsed time: 182.0902 seconds


### Nettoyage du dataset :

In [3]:
timer.start()

title_principals = title_principals
title_principals = title_principals[['nconst']][title_principals['category'].str.contains("actor|actress", regex=True)]

title_ratings = title_ratings.dropna()

title_basics = title_basics.dropna()
title_basics = title_basics[['titleType']][title_basics.titleType.str.contains('movie|tvMovie', regex=True)]

name_basics = name_basics.dropna()
name_basics = name_basics[['primaryName']][name_basics.primaryProfession.str.contains('actor|actress', regex=True)]

timer.stop("Nettoyage des datasets :")

Nettoyage des datasets :
Elapsed time: 114.9794 seconds


### Jointure des tables et calcul des moyennes :

In [4]:
actors_ranking = title_principals.merge(title_basics, left_index=True, right_index=True, how='left').drop(columns='titleType').dropna()
actors_ranking = actors_ranking.merge(name_basics, left_on='nconst', right_index=True, how='left').dropna()
actors_ranking = actors_ranking.merge(title_ratings, left_index=True, right_on=['tconst'], how='left')[['averageRating', 'primaryName']].dropna()
actors_ranking=actors_ranking.groupby('primaryName').agg({'averageRating': 'mean'})

### Import en CSV du fichier

In [5]:
actors_ranking.to_csv('rating_actors.csv', index=False)